# Lecture notes for 2020-04-08

## Direct to Iterative

For the first part of the semester, we discussed *direct* methods
for solving linear systems and least squares problems. These methods
typically involve a factorization, such as LU or QR, that reduces the
problem to a triangular solve using forward or backward substitution.
These methods run to completion in a fixed amount of time, and are
backed by reliable software in packages like LAPACK or UMFPACK.

There are a few things you need to know to be an informed *user*
(not developer) of direct methods:

-   You need some facility with matrix algebra, so that you know how to
    manipulate matrix factorizations and “push parens” in order to
    compute efficiently.

-   You need to understand the complexity of different factorizations,
    and a little about how to take advantage of common matrix structures
    (e.g. low-rank structure, symmetry, orthogonality, or sparsity) in
    order to effectively choose between factorizations and algorithms.

-   You need to understand a little about conditioning and the
    relationship between forward and backward error. This is important
    not only for understanding rounding errors, but also for
    understanding how other errors (such as measurement errors) can
    affect a result.

It’s also immensely helpful to understand a bit about how the methods
work in practice. On the other hand, you are unlikely to have to build
your own dense Gaussian elimination code with blocking for efficiency;
you’ll probably use a library routine instead. It’s more important that
you understand the ideas behind the factorizations, and how to apply
those ideas to use the factorizations effectively in applications.

Compared to direct methods, iterative methods provide more room for
clever, application-specific twists and turns. An iterative method for
solving the linear system $Ax = b$ produces a series of guesses
$$
  \hat{x}^1, \hat{x}^2, \ldots \rightarrow x.
$$
The goal of the iteration is not always to get the exact answer as fast as possible; it
is to get a good enough answer, fast enough to be useful. The rate at
which the iteration converges to the solution depends not only on the
nature of the iterative method, but also on the structure in the
problem. The picture is complicated by the fact that different
iterations cost different amounts per step, so a “slowly convergent”
iteration may in practice get an adequate solution more quickly than a
“rapidly convergent” iteration, just because each step in the slowly
convergent iteration is so cheap.

As with direct methods, though, sophisticated iterative methods are
constructed from simpler building blocks. In this lecture, we set up one
such building block: stationary iterations.

In [ ]:
using LinearAlgebra
using Plots

## Stationary Iterations

A stationary iteration for the equation $Ax = b$ is typically associated
with a *splitting* $A = M-N$, where $M$ is a matrix that is easy to
solve (i.e. a triangular or diagonal matrix) and $N$ is everything else.
In terms of the splitting, we can rewrite $Ax = b$ as 
$$
  Mx = Nx + b,
$$
which is the fixed point equation for the iteration
$$
  Mx^{k+1} = Nx^{k} + b.
$$
We can equivalently write this iteration in terms of $M$ and $A$
$$
  x^{k+1} = x^k + M^{-1} (b-Ax^k).
$$
If we subtract the fixed point equation from
the iteration equation, we have the error iteration
$$
  M e^{k+1} = N e^k
$$
or
$$
  e^{k+1} = R e^k, \quad R = M^{-1} N.
$$
Note: This is exactly the same thing we saw with power iteration,
except that we are not normalizing at every step!

We’ve already seen one example of such an iteration (iterative refinement with
an approximate factorization); in other cases, we might choose $M$ to be
the diagonal part of $A$ (Jacobi iteration) or the upper or lower
triangle of $A$ (Gauss-Seidel iteration). We will see in the next
lecture that there is an alternate “matrix-free” picture of these
iterations that makes sense in the context of some specific examples,
but for analysis it is often best to think about the splitting picture.

Let's see how we would actually implement something like this.  Usually, we
don't care about forming $M$ explicitly, we just want a way to do solves with
it.  Let's write the code to reflect that.  We'll provide implementations of
solves for both Jacobi and Gauss-Seidel.

In [ ]:
function jacobi_Msolve(A)
    d = diag(A)
    return (x) -> x ./ d
end

function gs_Msolve(A)
    L = tril(A)
    return (x) -> L \ x
end

function stationary(A, b, Msolve; rtol=1e-6, nsteps=100, monitor=(x, rnorm) -> nothing)
    x = Msolve(b)
    for k = 1:nsteps
        r = b-A*x
        rnorm = norm(r)
        monitor(x, rnorm)
        if rnorm < rtol * norm(b)
            return x
        end
        x += Msolve(r)
    end
    return x
end

Now let's look at a simple test matrix, and see if we get the right rate of convergence.
We know from our analysis of the power iteration that the convergence ought to look
something like
$$
  \|e^{k+1}\| \approx \rho(R) \|e^k\|,
$$
where $\rho(R)$ is the *spectral radius*
$$
  \rho(R) = \max_{\lambda \in \Lambda(R)} |\lambda|.
$$

In [ ]:
A = [4. 1. 0. ;
     1. 4. 1. ;
     0. 1. 4. ]
Msolve = jacobi_Msolve(A)

b = rand(3)
xref = A\b

Mjacobi = diagm(diag(A))
Rjacobi = Matrix{Float64}(I,3,3)-Mjacobi\A
lambda_max = maximum(abs.(eigvals(Rjacobi)))

rhist = Array{Float64,1}([])
stationary(A, b, Msolve, monitor=(x, rnorm) -> push!(rhist, rnorm))

println("Supposed rate constant: $lambda_max")
println("Empirical rate constant: $(rhist[end]/rhist[end-1])")

plot(rhist, yscale=:log10, legend=false)
plot!(lambda_max.^(1:length(rhist)), linestyle=:dash)

## Convergence: Norms and Eigenvalues

We consider two standard approaches to analyzing the convergence of a
stationary iteration, both of which revolve around the error iteration
matrix $R = M^{-1} N$. These approaches involve taking a norm inequality
or using an eigenvalue decomposition. The first approach is often easier
to reason about in practice, but the second is arguably more
informative.

For the norm inequality, note that if $\|R\| < 1$ for some operator
norm, then the error satisfies
$$
  \|e^{k+1}\| \leq \|R\| \|e^k\| \leq \|R\|^{k+1} \|e^0\|.
$$ Because
$\|R\|^k$ converges to zero, the iteration eventually converges. As an
example, consider the case where $A$ is *strictly row diagonally
dominant* (i.e. the sum of the magnitudes of the off-diagonal
elements in each row are less than the magnitude of the diagonal
element), and let $M$ be the diagonal part of $A$ (Jacobi iteration). In
that case, $\|R\|_\infty = \|M^{-1} N\|_\infty < 1$. Therefore, the
infinity norm of the error is monontonically decreasing.

Note that in finite-dimensional spaces, there is a property of “equivalence
of norms” that says that convergence in one norm implies convergence
in any other norm; however, this does *not* mean that monotone
convergence in one norm implies monotone convergence in any other
norm.

The $A$ matrix that we saw above is diagonally dominant, so let's look
at that as an example.

In [ ]:
rhist = Array{Float64,1}([])
stationary(A, b, Msolve, monitor=(x, rnorm) -> push!(rhist, norm(b-A*x, Inf)))
lambda_max2 = maximum(abs.(eigvals(Rjacobi)))

println("Spectrial radius: $lambda_max2")
println("Norm of R: $(opnorm(Rjacobi, Inf))")

plot(rhist, yscale=:log10, legend=false)
plot!(opnorm(Rjacobi, Inf).^(1:length(rhist)), linestyle=:dash)
plot!(lambda_max2.^(1:length(rhist)), linestyle=:dash, color=:blue)

As we can see from the plot, the convergence estimate we get by looking
at $\|R\|_{\infty}$ may be rather loose.

Bounding by one the infinity norm (or two norm, or one norm) of the
iteration matrix $R$ is *sufficient* to guarantee convergence, but
not *necessary*. In order to completely characterize when stationary
iterations converge, we need to turn to an eigenvalue decomposition.
Suppose $R$ is diagonalizable, and write the eigendecomposition as
$$
  R = V \Lambda V^{-1}.
$$
Now, note that $R^k = V \Lambda^k V^{-1}$, and therefore
$$
  \|e^k\| = \|R^k e^0\| = \|V \Lambda^k V^{-1} e^0\| \leq \kappa(V) \rho(R)^k \|e^0\|,
$$
where $\rho(R)$ is the *spectral radius* of
$R$, i.e. $$\rho(R) = \max_{\lambda \mbox{ an eig}} |\lambda|,$$ and
$\kappa(V) = \|V\| \|V^{-1}\|$. For a diagonalizable matrix, convergence
of the iteration happens if and only if the spectral radius of $R$ is
less than one. *But* that statement ignores the condition number of
the eigenvector matrix! For highly “non-normal” matrices in which the
condition number is large, the iteration may appear to make virtually no
progress for many steps before eventually it begins to converge at the
rate predicted by the spectral radius. This is consistent with the
bounds that we can prove, but often surprises people who have not seen
it before.

#### Questions

1.  What is $\|R\|_\infty$ for the example used above?  How does it compare to $\rho(R) \approx 0.35$?

*Answer*:

2.  In general, $\rho(R) \leq \|R\|$ for any operator norm.  Why?

*Answer*: 

## A model problem

For the sake of concreteness in our discussion of iterative linear solvers,
let’s consider a standard model problem: a
discretization of a Poisson equation on a line. That is, we approximate
$$
  -\frac{d^2 u}{dx^2} = f, \quad u(0) = u(1) = 0
$$
using the second-order finite difference approximation
$$\frac{d^2 u}{dx^2} \approx \frac{u(x-h)-2u(x)+u(x+h)}{h^2}$$ where $h$
is a small step size. We discretize the problem by meshing $[0,1]$ with
evenly spaced points $x_j = jh$ for $j = 0$ to $N+1$ where
$h = 1/(N+1)$, then apply this approximation at each point. This
procedure yields the equations
$$
  -u_{j-1}+2u_j-u_{j+1} = h^2 f_j, \quad j = 1, \ldots, N
$$
We can write this in matrix form as an $N \times (N+2)$ linear systems
$$
  \begin{bmatrix}
   -1 & 2 & -1 \\
      & -1 & 2 & -1 \\
      &    & \ddots & \ddots & \ddots \\
      &    &        & -1 & 2 & -1 \\
      &    &        &    & -1 & 2 & -1
  \end{bmatrix}
  \begin{bmatrix} u_0 \\ u_1 \\ u_2 \\ \vdots \\ u_{N-1} \\ u_N \\ u_{N+1} \end{bmatrix} =
  h^2 \begin{bmatrix} f_1 \\ f_2 \\ \vdots \\ f_{N-1} \\ f_N \end{bmatrix},
$$
or, in matrix form,
$$
  \bar{T} \bar{u} = h^2 f
$$
where $\bar{T} \in \mathbb{R}^{N \times (N+2)}$, $\bar{u} \in \mathbb{R}^{N+2}$ is the vector of 
values $u_0, \ldots, u_{N+1}$ (including at the boundaries) and $f \in \mathbb{R}^N$ denotes the 
vector of values  $f_1, \ldots, f_N$ (only on interior points).  In order to get a square system,
we use the boundary conditions on $u_0$ and $u_{N+1}$
to move those pieces of the system to the right hand side
$$
  \begin{bmatrix}
    2 & -1 \\
   -1 & 2 & -1 \\
      & \ddots & \ddots & \ddots \\
      &        & -1 & 2 & -1 \\
      &        &    & -1 & 2
  \end{bmatrix}
  \begin{bmatrix} u_1 \\ u_2 \\ \vdots \\ u_{N-1} \\ u_N \end{bmatrix} =
  h^2 \begin{bmatrix} f_1 \\ f_2 \\ \vdots \\ f_{N-1} \\ f_N \end{bmatrix} 
  - u_0 \begin{bmatrix} -1 \\ 0 \\ \vdots \\ 0 \\ 0 \end{bmatrix}
  - u_{N+1} \begin{bmatrix} 0 \\ 0 \\ \vdots \\ 0 \\ -1 \end{bmatrix}.
$$
In our case, the boundary conditions are homogeneous ($u_0 = u_{N+1} = 0$), so
in matrix notation we have $$Tu = h^2 f$$ where $u$ and $f$ are vectors in
${\mathbb{R}}^N$ representing the sampled (approximate) solution and the
sampled forcing function on the interior mesh points.
The matrix $T$ is a frequently-recurring model matrix, the tridiagonal 
$$
  T =
  \begin{bmatrix}
   2 & -1 \\
  -1 &  2 & -1 \\
     & -1 &  2 & -1 \\
     &    & \ddots & \ddots & \ddots \\
     &    &        & -1 & 2 & -1 \\
     &    &        &    & -1 & 2
  \end{bmatrix}.
$$

In [ ]:
formT(N) = SymTridiagonal(2*ones(N), -ones(N-1))
Ngrid = 11
T = formT(Ngrid)

In [ ]:
# Get a vector of grid coordinates and spacing parameter
xsall = range(0.0, stop=1.0, length=Ngrid+2)
xs = xsall[2:end-1]
h = 1/(Ngrid+1)

# Form a right hand side and solve the PDE
fs = 0.5 .- abs.(xs.-0.5)
us = T\(h^2*fs)

# Plot the right hand side and u
l = @layout [a b]    
p1 = plot(xs, fs, label="f(x)")
p2 = plot(xs, us, label="u(x)")
plot(p1, p2, layout=l)

## Splitting and sweeping

Splitting is the right linear algebraic framework for discussing
convergence of stationary methods, but it is not the way they are
usually programmed. The connection between a matrix splitting and a
“sweep” of a stationary iteration like Gauss-Seidel or Jacobi iteration
is not always immediately obvious, and so it is probably worth spending
a moment or two explaining in more detail.

Suppose we forgot about how cheap Gaussian elimination is for
tridiagonal matrices. How might we solve this system of equations? A
natural thought is that we could make an initial guess at the solution,
then refine the solution by “sweeping” over each node $j$ and adjusting
the value at that node ($u_j$) to be consistent with the values at
neighboring nodes. In one sweep, we might compute a new set of values
$u^{\mathrm{new}}$ from the old values $u^{\mathrm{old}}$, 
or we might update the values for each node in turn, using the most
recent estimate for each update.  These are, respectively, a step of
*Jacobi* iteration and a step of *Gauss-Seidel* iteration, which are two
standard stationary methods.

It's convenient to do this with the full versions of the vectors (i.e.
including the boundary values).  Julia being Julia, we could define a
new type to make it simpler to iterate over interior points alone,
but we will not bother to do so here.

In [ ]:
function jacobi_sweep!(unew, uold, f, h)
    Ngrid= length(unew)-2
    for jj = 1:Ngrid
        j = jj+1  # Adjust offset since Julia array default to 1-based indexing
        unew[j] = (h^2*f[j] + uold[j+1] + uold[j-1])/2
    end
end

function gs_sweep!(u, f, h)
    Ngrid = length(unew)-2
    for jj = 1:Ngrid
        j = jj+1
        u[j] = (h^2*f[j] + u[j+1] + u[j-1])/2
    end
end

For this model problem, each Gauss-Seidel step gives us the same benefit as about two
Jacobi steps.

In [ ]:
fall = [0; fs; 0]
uref = [0; us; 0]
uhat = [0; h^2*fs; 0]
ugs  = copy(uhat)
unew = zeros(Ngrid+2)

err_j  = zeros(100)
err_gs = zeros(100)

anim = @animate for sweep = 1:100
    jacobi_sweep!(unew, uhat, fall, h)
    gs_sweep!(ugs, fall, h)
    err_j[sweep] = norm(unew-uref)
    err_gs[sweep] = norm(ugs-uref)
    plot(xsall, uref, linewidth=2, color=:black, label="Reference solution")
    plot!(xsall, unew, linecolor=:blue, label="Jacobi")
    plot!(xsall, ugs, linecolor=:red, label="Gauss-Seidel")
    uhat[:] = unew
end
gif(anim, fps=10)

In [ ]:
plot(err_j, yscale=:log10, linecolor=:blue, xlabel="Sweeps", ylabel="Error", label="Jacobi")
plot!(err_gs, yscale=:log10, linecolor=:red, label="Gauss-Seidel")

How should we relate the “sweep” picture to a matrix splitting? The
update equation from step $k$ to step $k+1$ in Jacobi is
$$
  -u_{j-1}^{(k)}+2u_j^{(k+1)}-u_{j+1}^{(k)} = h^2 f_j,
$$
while the Gauss-Seidel update is
$$
  -u_{j-1}^{(k+1)}+2u_j^{(k+1)}-u_{j+1}^{(k)} = h^2 f_j.
$$
In terms of splittings, this means that Jacobi corresponds to taking $M$ to be the
diagonal part of the matrix, 
$$
  M =
  \begin{bmatrix}
   2 &    \\
     &  2 &    \\
     &    &  2 &    \\
     &    &        & \ddots &  \\
     &    &        &    & 2 &  \\
     &    &        &    &   & 2
  \end{bmatrix}, ~~
  N =
  \begin{bmatrix}
   0 &  1 \\
   1 &  0 &  1 \\
     &  1 &  0 & 1 \\
     &    & \ddots & \ddots & \ddots \\
     &    &        & 1 & 0 & 1 \\
     &    &        &   & 1 & 0
  \end{bmatrix},
$$ while Gauss-Seidel corresponds to taking $M$ to be the
lower triangle of the matrix, 
$$
  M =
  \begin{bmatrix}
   2 &  \\
  -1 &  2 &  \\
     & -1 &  2 &  \\
     &    & \ddots & \ddots &  \\
     &    &        & -1 & 2 &  \\
     &    &        &    & -1 & 2
  \end{bmatrix}, ~~
  N =
  \begin{bmatrix}
   0 &  1 \\
     &  0 &  1 \\
     &    &  0 & 1 \\
     &    &    & \ddots & \ddots \\
     &    &        &  & 0 & 1 \\
     &    &        &    &   & 0
  \end{bmatrix}.
$$

The point of this exercise is that *programming* stationary
iterative methods and *analyzing* the same methods may lead
naturally to different ways of thinking about the iterations. It’s
worthwhile practicing mapping back and forth between these two modes of
thought.

#### Questions

1.  Write a code to plot the convergence of Jacobi and Gauss-Seidel in the case when the right hand side $f$ is zero, but the boundary conditions are $u_0 = 0$ and $u_{N+1} = 1$.

2.  Based on the experiment above, do you have any comments on how you expect the convergence rate to behave as a function of $N$?

*Answer*:

## Linear Solves and Quadratic Minimization

We have already briefly described an argument that Jacobi iteration
converges for strictly row diagonally dominant matrices. We now discuss
an argument that Gauss-Seidel converges (or at least part of such an
argument). In the process, we will see a useful way of reformulating the
solution of symmetric positive definite linear systems that will prepare
us for our upcoming discussion of conjugate gradient methods.

Let $A$ be a symmetric positive definite matrix, and consider the
“energy” function $$\phi(x) = \frac{1}{2} x^T A x - x^T b.$$ The
stationary point for this function is the point at which the derivative
in any direction is zero. That is, for any direction vector $u$,
$$\begin{aligned}
  0
  &=\left. \frac{d}{d\epsilon} \right|_{\epsilon = 0} \phi(x+\epsilon u) \\
  &=\frac{1}{2} u^T A x + \frac{1}{2} x^T A u -  u^T b \\
  &=u^T (Ax-b)
\end{aligned}$$
Except in pathological instances, a
directional derivative can be written as the dot product of a direction
vector and a gradient; in this case, we have 
$$
  \nabla \phi = Ax-b.
$$
Hence, minimizing $\phi$ is equivalent to solving $Ax = b$.

Now that we have a characterization of the solution of $Ax = b$ in terms
of an optimization problem, what can we do with it? One simple approach
is to think of a sweep through all the unknowns, adjusting each variable
in term to minimize the energy; that is, we compute a correction
$\Delta x_j$ to node $j$ such that
$$\Delta x_j = \operatorname{argmin}_z \phi(x+z e_j)$$ Note that
$$\frac{d}{dz} \phi(x+z e_j) = e_j^T (A(x+ze_j)-b),$$ and the update
$x_j := x_j + \Delta x_j$ is equivalent to choosing a new $x_j$ to set
this derivative equal to zero. But this is exactly what the Gauss-Seidel
update does! Hence, we can see Gauss-Seidel in two different ways: as a
stationary method for solving a linear system, or as an optimization
method that constantly makes progress toward a solution that minimizes
the energy. The latter perspective can be turned (with a little
work) into a convergence proof for Gauss-Seidel on positive-definite
linear systems.